In [2]:
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
import pandas as pd 
import pickle
from torch.utils.data import DataLoader
import datetime
from sentence_transformers.losses import CoSENTLoss, MatryoshkaLoss
from sklearn.model_selection import train_test_split


/Users/mohammadrezaghassemzadeh/Desktop/p-matching/product-matching/bachelorarbeit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/ts/4_j3m5qs2wvgc9jjm_yb5p1h0000gn/T/ipykernel_1435/570986289.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_pickle(r'daten/otto_amazon_variation_pairs.pkl', compression='gzip')
df.head(2)

,amazon_product_name,otto_variation_name,variation_id,variation_asin,product_asin,amazon_product_url,file_name_timestamp,amazon_product_url_zyte,amazon_main_imageurl,amazon_description,...,pdh_variation_h_sk,pdh_product_h_sk,product_id,pbk,product_type,otto_product_url,otto_main_imageurl,kategorie_direct_bezeichnung,kategorie_bezeichnung,sub_kategorie_bezeichnung
0,Brother DCP-J1200W 3-in-1 Farbtintenstrahl-Mul...,"Brother DCP-J1200W Tintenstrahldrucker, (WLAN ...",1459585412,B0972RD6F5,B0972RD6F5,https://www.amazon.de/dp/B0972RD6F5/,2023-10-23 02:47:47+00:00,https://www.amazon.de/Brother-DCP-J1200W-Farbt...,https://m.media-amazon.com/images/I/41jxuc8HZs...,Produktbeschreibung des Herstellers ...,...,5d7afa8d68a6d88798dd37ed27093882,00b76adf49c4598d0a07b800e7958d36,1459585411,Drucker,Tintenstrahldrucker,https://otto.de/p/1459585411,https://i.otto.de/i/otto/bc4e3190-8618-54bf-b8...,Hardware,Multimedia & Toys,"Office Equipment, Mobile Computing & Digital I..."
1,AEG CCB544COBM 50 cm Standherd mit Glaskeramik...,AEG Elektro-Standherd CCB544COBM,1475311739,B0983HH8YJ,B0983HH8YJ,https://www.amazon.de/dp/B0983HH8YJ/,2023-11-10 01:13:44+00:00,https://www.amazon.de/AEG-CCB544COBM-Glaskeram...,https://m.media-amazon.com/images/I/71lHc-qSRT...,None,...,36643027eca54abd01e3d16fa0c3f313,0734c4795c54ca56c57c42174c79b9c2,1475296433,Herdsets,None,https://otto.de/p/1475296433,https://i.otto.de/i/otto/05f579c0-7391-5264-8e...,Hardware,Home Appliances,Built-In & Cooking


In [4]:
import pandas as pd
import random

def negative_examples(df, num_negatives_per_positive=1):
    """
    hierbei sind die Produkte ähnlich, aebr nicht identisch 
    :return: DataFrame mit den negativen Beispielen.
    """
    negative_examples = []
    categories = df['kategorie_bezeichnung'].unique()

    for _, row in df.iterrows():
        for _ in range(num_negatives_per_positive):

            current_category = row['kategorie_bezeichnung']
            
            # Produkte innerhalb der gleichen Kategorie, aber nicht das gleiche Produkt
            similar_products = df[(df['kategorie_bezeichnung'] == current_category) & 
                                  (df['amazon_product_name'] != row['amazon_product_name']) &
                                  (df['otto_variation_name'] != row['otto_variation_name'])]
            
            # Wenn es ähnliche Produkte gibt, wird eines zufällig als negatives Beispiel gewählt
            if not similar_products.empty:
                negative_example = similar_products.sample(1)
                product1 = row['amazon_product_name']
                product2 = negative_example['otto_variation_name'].values[0]
                
                negative_examples.append({'amazon_product_name': product1, 'otto_variation_name': product2})

    return pd.DataFrame(negative_examples)

In [5]:
df_negative = negative_examples(df, num_negatives_per_positive=1)

In [6]:
positive_examples = [InputExample(texts=[str(row['amazon_product_name']), str(row['otto_variation_name'])]) for index, row in df.iterrows()]

In [7]:
negative_examples = [InputExample(texts=[str(row['amazon_product_name']), str(row['otto_variation_name'])]) for index, row in df_negative.iterrows()]

In [8]:
output_dir = r'/Users/mohammadrezaghassemzadeh/Desktop/p-matching/product-matching/SentenceT/ausgabe'
base_model = 'all-MiniLM-L6-v2'
epochs = 2
model = SentenceTransformer(base_model)

In [9]:
all_examples = positive_examples + negative_examples

In [59]:
""" # für zeile in Tabele: amzon und otto in texte pacekn!!!!!!
train_examples = [
    InputExample(texts=[row['amazon_product_name'], row['otto_variation_name']])
    for _, row in df.iterrows()
] """

In [10]:
train_data, validation_data = train_test_split(all_examples, test_size=0.1)

In [11]:
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)
validation_dataloader = DataLoader(validation_data, shuffle=True, batch_size=32)

In [12]:
from sentence_transformers.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator.from_input_examples(validation_data, name='validation')

In [13]:
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [14]:
from sentence_transformers.losses import MatryoshkaLoss
loss = losses.MatryoshkaLoss(model=base_model, loss=train_loss, matryoshka_dims=[768, 512, 256, 128, 64])

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=epochs,
    evaluation_steps=500,
    use_amp=False
)

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
model_name = f"{base_model}-similar_otto_amazon_products-{timestamp}"
model.save(f"{output_dir}/{model_name}")
